# Exceptions

In this chapter, we will study the Python's exception mechanism. Any running program may be subject to errors for which detection and fallback strategies are possible. These errors are not *necessarily* bugs but exceptional conditions (or **exceptions**) in the normal course of part of a program. For example, the absence of a useful file is not a bug in the program; however, not dealing with its absence would cause one. In systems without exceptions, routines would need to return some special error code. However, this is sometimes complicated as users of the routine need to write extra code to distinguish normal return values from erroneous ones. On the other side, the exception mechanism allows you to manage this kind of exceptional conditions during program execution. When an exception occurs, normal program execution is interrupted and the exception is processed.

The exception mechanism relies on two ingredients: exception handlers & exception reporting:
 - An exception handler establishes a set of error handling routines, defined by the programmer, on a block (in a function or method of the program). These routines are activated for the entire execution time of the protected block. The notion of execution of the protected block includes the whole chain of calls (of functions or methods) from this block: the exception handlers are said to be active within the dynamic range of the block.
 - When an error condition is detected, it is said to be signaled: an error handling block (a handler) is searched for in the list of active handlers. The execution of the program is deferred to the processing block. Exception reporting can be automatic, if it corresponds to an exception defined in the programming language or a provided library, or it can be triggered by the programmer by using a dedicated primitive. It is possible to construct new types of exceptions. It may be that no handler has been provided by the programmer, in which case a default handler is selected.
 
When you execute Python code in an interpreter such as python3, any exceptions that propagate to the top are then captured by Python itself and transformed into an error message. With no doubt, you have encountered such error messages yourself:

In [1]:
1/0 # Dividing by 0

ZeroDivisionError: division by zero

In [2]:
[0, 1][2] # Accessing the third element of a 2-element list

IndexError: list index out of range

These messages tell you that some error conditions (exceptions) have been raised and not handled correctly. The error condition comes with some information --- you can see that the error message is different between the two examples. The first one tells that a division by `0` has been executed, while the second one tells that the program encountered an out-of-bound access to a list.

We are now going to see how to handle these error conditions and how to raise and define new exceptions.


## The `try...except...` block

The `try...except...` construction is the core instruction for installing an error condition handler. In the following example, the code `1/0` (this is going to trigger an error) is protected using the `try...except...` instruction.

In [3]:
try:
    print('entering "try"')
    1/0
    print('leaving "try"')
except:
    print('in "except" block')
print('continuing')

entering "try"
in "except" block
continuing


During the execution of the code above, the control flow is the following: with first enter the `try` block, following the normal execution flow - we can see this because the program prints `entering "try"`. However, when executing `0/1`, an exception is raised. In that case, the program is interrupted and the nearest `except` block (in the call chain) is searched for (here the block is just below) and the control is then given to that block. This results in `leaving "try"` not being printed - we have `in "except" block` instead. At the end of the `except` block, the program execution continues linearly, moving to the next instruction after the `try...exept...` block.

Is it important to understand that the `except` block is only executed when an exception is raised in the `try` block. For instance, in the following example, the `except` block won't be executed because the `try` one does not raise any exception:

In [4]:
try:
    print('entering "try"')
    print('leaving "try"')
except:
    print('in "except" block')
print('continuing')

entering "try"
leaving "try"
continuing


In Python, some data is attached to an error condition. This data must be a instance of the class `Exception` and any of its descendant. For example, in the introductory examples, we saw two of them: `ZeroDivisionError` and `IndexError`. It is possible to get access to this error condition data in the `except` block, as shown in the following example:

In [5]:
help(Exception)

Help on class Exception in module builtins:

class Exception(BaseException)
 |  Common base class for all non-exit exceptions.
 |  
 |  Method resolution order:
 |      Exception
 |      BaseException
 |      object
 |  
 |  Built-in subclasses:
 |      ArithmeticError
 |      AssertionError
 |      AttributeError
 |      BufferError
 |      ... and 15 other subclasses
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from BaseException:
 |  
 |  __delattr__(self, name, /)
 |      Implement delattr(self, name).
 |  
 |  __getattribute__(self, name, /

In [6]:
try:
    print('entering "try"')
    1/0
    print('leaving "try"')
except Exception as e:
    # `e` is a variable containing the error condition data
    print(f'in "except" block: {type(e)}')
print('continuing')

entering "try"
in "except" block: <class 'ZeroDivisionError'>
continuing


For that, we simply write `except Exception as e` that tells the Python interpreter that we want to catch any exception whose data is an instance of `Exception` (or any of its subclasses) and that we want to access the exception data via the variable named `e`.

Note that we could be more precise by replacing `Exception` with a subclass of `Exception`, as we do in the following example:

In [7]:
try:
    print('entering "try"')
    1/0
    print('leaving "try"')
except ArithmeticError as e:
    # `e` is a variable containing the error condition data
    print('in "except" block: {}'.format(type(e)))
print('continuing')

entering "try"
in "except" block: <class 'ZeroDivisionError'>
continuing


In that case, the `except` block will only be registered by catching exceptions whose data is an instance of the subclass of `ZeroDivisionError`. In any other case, the block is going to be ignored. E.g., in the following code, the exception is going to escape up to the top-level:

In [8]:
try:
    print('entering "try"')
    [0, 1][2] # Raises `IndexError`
    print('leaving "try"')
except ZeroDivisionError as e:
    # `e` is a variable containing the error condition data
    print('in "except" block: {}'.format(type(e)))
print('continuing')

entering "try"


IndexError: list index out of range

Note that is it possible to install multiple exception handlers. In that case, in case of an error condition, the first block (from top to bottom) that matches the exception is executed (all other blocks are ignored - even if they are other blocks matching the exception data):

In [9]:
import math

try:
    print('entering "try"')
    math.log(0)
    print('leaving "try"')
except IndexError:
    # Does not match `ZeroDivisionError`
    print('exn: index error')
except ZeroDivisionError:
    # Does match `ZeroDivisionError`
    print('exn: zero division error')
except Exception as e:
    # Does match `ZeroDivisionError`...
    # ...but is not the first matching block
    print(f'exn: all other exceptions: {type(e)}')
print('continuing')

entering "try"
exn: all other exceptions: <class 'ValueError'>
continuing


Note that exceptions allow *non-local* jumps: when an exception is raised, the exception handlers are searched along the full call chains and the nearest matching block is executed. For example:

In [10]:
def myfunction():
    # The exception `ZeroDivisionError` is going to escape the function
    try:
        1/0
    except IndexError:
        # Does not match ZeroDivisionError
        print('index-error')
    print('at the end of myfunction')
        
try:
    myfunction()
except ZeroDivisionError:
    # This block is going to capture the exception raised
    # in the function `myfunction`.
    # We here have a non-local jump: we jump directly from the
    # `myfunction` body to that block.
    print('zero-division-error')
print('continuing')

zero-division-error
continuing


Also, note that this is the *nearest* block (in term of code depth and call stack) that is selected. When we execute the following code, we first enter `TRY-1`, then `TRY-2`, and then `TRY-3`. When an exception is raised in `TRY-3`, the except blocks are selected in the reverse order: the ones of `TRY-3`, then the ones of `TRY-2` and then the ones of `TRY-1`. As soon as a matching `except` block is found, it gets executed and the other ones are ignored.

In [11]:
def myfunction():
    try: # TRY-3
        1/0
    except ZeroDivisionError:
        print('zero-division-error 3')
    print('at the end of myfunction')
    
try: # TRY-1
    try: # TRY-2
        myfunction()
    except ZeroDivisionError:
        print('zero-division-error 2')
except ZeroDivisionError:
    print('zero-division-error 1')
print('continuing')

zero-division-error 3
at the end of myfunction
continuing


## `finally` blocks

It is possible to add a last block to the `try...except...` block that is going to get executed at the end of the block regardless of wether an exception has been triggered or not. This block is added at the end and is prefixed by the `finally` keywords, as shown in the following example:

In [12]:
try:
    print('entering "try"')
    1/0
    print('leaving "try"')
except IndexError:
    # Does not match `ZeroDivisionError`
    print('exn: index error')
except ZeroDivisionError:
    # Does match `ZeroDivisionError`
    print('exn: zero division error')
except Exception:
    # Does match `ZeroDivisionError`...
    # ...but is not the first matching block
    print('exn: all other exceptions')
finally:
    print('finally...')
    
print('continuing')

try:
    print('second block')
except Exception:
    print('second exn')
finally:
    print('second finally')
print('second continuing')

try:
    1/0
    print('bla')
finally:
    print('third finally')

entering "try"
exn: zero division error
finally...
continuing
second block
second finally
second continuing
third finally


ZeroDivisionError: division by zero

Such a block allows to, e.g., deallocate resources - an operation that should be done regardless of the existence of an error condition. E.g., when opening a file, the file should always be closed when you are done with it:

In [13]:
stream = None
try:
    stream = open('myfile.txt', 'r')
    # do something with stream
except IOError as e:
    # we had an I/O error while reading from stream...
    # ...we might want to do something here
    pass
finally:
    # But in all cases, we have to close that file
    if stream is not None:
        stream.close()
    stream = None

If a not handled exception is raised in the `try` block, the `finally` block gets executed but at the end, the exception is raised again. E.g., in the following example, we are going to print `in finally` in all cases, but since the exception `ZeroErrorDivision` in not caught, it will escape:

In [14]:
try:
    1/0
finally:
    print('in finally')

in finally


ZeroDivisionError: division by zero

Also, note that the `except` and `finally` blocks are not protected by the associated `try` block. For example, if you raise an exception in a `except` block, it is going to escape the current `try...except...` block. However, you can see that Python keeps a track of this. E.g., in the following example, it tells you that an `IndexError` has been triggered while handling the `ZeroDivisionError` error condition:

In [15]:
try:
    try:
        1/0
    except ZeroDivisionError:
        [0, 1][2]
except Exception as e:
    print(e)

list index out of range


In [16]:
try:
    1/0
finally:
    [0, 1][2]

IndexError: list index out of range

## Raising exceptions (the `raise` instruction)

So far, we have seen how to handle exceptions and examples of error conditions raised by the Python standard library or language. What we miss is a way to raise, in our code, exceptions. In Python, raising an exception is simply done using the `raise` keyword followed by an instance of the `Exception` class:

In [17]:
try:
    # The () are not mandatory...
    # ...if you write [raise ZeroDivisionError], Python is going
    # ...to create an instance of the class `ZeroDivisionError` for you
    # ...(in that case, the constructor must not have arguments).
    raise ZeroDivisionError()
except ZeroDivisionError:
    print('oups')

oups


You can also create your own exceptions by subclassing `Exception` or any descendant of `Exception`. For example:

In [18]:
class MyException(Exception):
    pass

class MyException2(MyException):
    pass

try:
    raise MyException2
except ZeroDivisionError:
    print('hu?')
except MyException:
    print('my exception')

my exception


You can see that, as excepted we executed the second `except` block (the first one is attached to the exception `ZeroDivisionError` that is not a subclass of `MyException`).

It is perfectly fine to define data attributes in exceptions. In that case, you can access these attributes in the `except` block:

In [19]:
class MyExceptionWithData(Exception):
    def __init__(self, data):
        super().__init__()
        self.data = data

try:
    raise MyExceptionWithData(42)
except MyExceptionWithData as e:
    print('my exception: {}'.format(e.data))

my exception: 42


## Context managers

<div style="border: 1px solid black; background-color: #ffcccc; margin: 10px 0; padding: 5px;">
This part is optional.
</div>

Context managers are a Python construction that allows to run code when we exit a block, regardless of wether an exception occured or not. We saw that such a behaviour can be obtained using the `try...finally...` construction:

In [20]:
stream = None
try:
    stream = open('myfile.txt', 'r')
    # do something with stream
except IOError as e:
    # we had an I/O error while reading from stream...
    # ...we might want to do something here
    pass
finally:
    # But in all cases, we have to close that file
    if stream is not None:
        stream.close()

However, context managers allow for a more elegant and more readable way to write such code. It also allows to define, once and for all, how a resource should be teared down. For instance, the code above could be rewritten as follow:

In [21]:
try:
    with open('myfile.txt', 'r') as stream:
        # do something with stream
        pass
except IOError as e:
    # we had an I/O error while reading from stream...
    # ...we might want to do something here
    pass

Internally, a context manager is any object that exposes two methods: `.__enter__()` and `.__exit__()`. Context managers are accessed using the `with expr as v` construction. When entering a `with` block, the interpreter calls the `.__enter__()` method of `expr`. This method can optionally return a value that represents the resource - in that case, the variable `v` is bound to that value. When exiting the `with` block, the resource is being disposed by a call to the `.__exit__()` method - regardless of wether we exit the block normally or not.

You can define your own context managers. For example:

In [22]:
class MyContextManager:
    def __enter__(self):
        print("entering...")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        # The methods is taking extra-arguments describe the exception that caused the
        # context to be exited (or None if the context exited normally)
        print("...exiting")
        
with MyContextManager() as x:
    print('in body')

entering...
in body
...exiting
